In [1]:
# This code will attempt to replicate the Fama French Momentum Factor aka WML (Winners Minus Losers).

In [2]:
import MalekFinance as mf
import pandas as pd
import numpy as np
import datetime as dt
from pandas.tseries.offsets import MonthEnd
from matplotlib import pyplot as plt
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 14)
pd.set_option('mode.chained_assignment', None)

In [3]:
formation_dates = pd.date_range('1965-12-31','2022-10-31',freq='M')
returns_dates = pd.date_range('1966-2-28','2022-12-31',freq='M')
data = mf.read('WRDS Data','CRSP Returns Data All Exchanges 63-22')
data.date = pd.to_datetime(data.date,dayfirst=True)
data = data[data['EXCHCD'].isin([1,2,3])]
data

,PERMNO,date,EXCHCD,TICKER,PRIMEXCH,RET
0,10000,1986-01-07,3,OMFGA,Q,C
1,10000,1986-01-08,3,OMFGA,Q,-0.024390
2,10000,1986-01-09,3,OMFGA,Q,0.000000
3,10000,1986-01-10,3,OMFGA,Q,0.000000
4,10000,1986-01-13,3,OMFGA,Q,0.050000
...,...,...,...,...,...,...
91828866,93436,2022-12-23,3,TSLA,Q,-0.017551
91828867,93436,2022-12-27,3,TSLA,Q,-0.114089
91828868,93436,2022-12-28,3,TSLA,Q,0.033089
91828869,93436,2022-12-29,3,TSLA,Q,0.080827


In [4]:
NYSE = data[data['EXCHCD'] == 1]
NYSEPERMNO = NYSE['PERMNO'].copy()
data = data.drop(['PRIMEXCH'],axis=1)
data = data.drop(['EXCHCD'],axis=1)
data.date = pd.to_datetime(data.date,dayfirst=True)
data = data[~data.duplicated(subset=['PERMNO', 'date'], keep='first')]
data1 = data.pivot(index='date', columns='PERMNO', values='RET')
data1.columns = data1.columns.astype('int')
data1 = data1.apply(pd.to_numeric, errors='coerce')
monthly_returns = data1.resample('M').agg(lambda x: (x+1).prod()-1)
monthly_returns.replace({0:np.nan},inplace=True)
past_11 = (monthly_returns+1).rolling(11).apply(np.prod)-1
monthly_returns

PERMNO,10000,10001,10002,10003,10004,10005,10006,...,93430,93431,93432,93433,93434,93435,93436
date,,,,,,,,,,,,,,,
1963-01-31,NaN,NaN,NaN,NaN,NaN,NaN,0.047003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-02-28,NaN,NaN,NaN,NaN,NaN,NaN,0.038282,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-03-31,NaN,NaN,NaN,NaN,NaN,NaN,-0.009009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-04-30,NaN,NaN,NaN,NaN,NaN,NaN,0.084848,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-05-31,NaN,NaN,NaN,NaN,NaN,NaN,0.091935,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.347369,NaN,-0.072489
2022-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.429688,NaN,-0.037591
2022-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.356165,NaN,-0.142169


In [5]:
data_size = mf.read('WRDS Data','CRSP Size Data All Exchanges 63-22')
data_size.date = pd.to_datetime(data_size.date,dayfirst=True)
data_size['Market Cap'] = (data_size['PRC']/data_size['CFACPR'])*(data_size['SHROUT']*data_size['CFACSHR'])
data_size = data_size[~data_size.duplicated(subset=['PERMNO', 'date'], keep='first')]
data_size1 = data_size.pivot(index='date', columns='PERMNO', values='Market Cap')
data_size1 = data_size1.abs()
data_size1 = data_size1.apply(pd.to_numeric,errors='coerce')
monthly_size = data_size1.resample('M').last()
monthly_size*=1000
NYSE_Size_df = monthly_size[monthly_size.columns.intersection(NYSEPERMNO)]
NYSE_Past_11_df = past_11[past_11.columns.intersection(NYSEPERMNO)]
Returns_Check_NYSE_df = monthly_returns[monthly_returns.columns.intersection(NYSEPERMNO)]
monthly_size

PERMNO,10000,10001,10002,10003,10004,10005,10006,...,93430,93431,93432,93433,93434,93435,93436
date,,,,,,,,,,,,,,,
1963-01-31,NaN,NaN,NaN,NaN,NaN,NaN,117329750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-02-28,NaN,NaN,NaN,NaN,NaN,NaN,120962250.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-03-31,NaN,NaN,NaN,NaN,NaN,NaN,119872500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-04-30,NaN,NaN,NaN,NaN,NaN,NaN,130043500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-05-31,NaN,NaN,NaN,NaN,NaN,NaN,140941000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,54507520.0,NaN,8.636156e+11
2022-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,31103840.0,NaN,8.376595e+11
2022-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,42196770.0,NaN,7.185149e+11


# Simulation

In [6]:
def WML_VW(formation,offset):
    NYSE_Size = NYSE_Size_df.loc[formation].dropna()
    NYSE_Past_11 = NYSE_Past_11_df.loc[formation].dropna()
    Returns_Check_NYSE = Returns_Check_NYSE_df.loc[formation + MonthEnd(offset)].dropna()
    NYSE_Merged = pd.concat([Returns_Check_NYSE,NYSE_Past_11,NYSE_Size],axis=1).dropna()
    NYSE_Merged.columns = ['2 Month Forward Return','Past 11','Market Cap']
    Winners_Threshold = NYSE_Merged['Past 11'].quantile(0.7)
    Losers_Threshold = NYSE_Merged['Past 11'].quantile(0.3)
    Size_Breakpoint = NYSE_Merged['Market Cap'].median()
    All_Size = monthly_size.loc[formation].dropna()
    All_Past_11 = past_11.loc[formation].dropna()
    All_Returns = monthly_returns.loc[formation + MonthEnd(offset)].dropna()
    All_Merged = pd.concat([All_Returns,All_Past_11,All_Size],axis=1).dropna()
    All_Merged.columns = ['2 Month Forward Return','Past 11','Market Cap']
    Large = All_Merged[All_Merged['Market Cap'] >= Size_Breakpoint]
    Small = All_Merged[All_Merged['Market Cap'] <= Size_Breakpoint]
    Large_Winners = Large[Large['Past 11'] >= Winners_Threshold]
    Large_Losers = Large[Large['Past 11'] <= Losers_Threshold]
    Small_Winners = Small[Small['Past 11'] >= Winners_Threshold]
    Small_Losers = Small[Small['Past 11'] <= Losers_Threshold]
    Large_Winners_RET = sum(Large_Winners.iloc[:,0] * (Large_Winners.iloc[:,2]/sum(Large_Winners.iloc[:,2])))
    Large_Losers_RET = sum(Large_Losers.iloc[:,0] * (Large_Losers.iloc[:,2]/sum(Large_Losers.iloc[:,2])))
    Small_Winners_RET = sum(Small_Winners.iloc[:,0] * (Small_Winners.iloc[:,2]/sum(Small_Winners.iloc[:,2])))
    Small_Losers_RET = sum(Small_Losers.iloc[:,0] * (Small_Losers.iloc[:,2]/sum(Small_Losers.iloc[:,2])))
    Portfolio_Return = ((Large_Winners_RET + Small_Winners_RET) - (Large_Losers_RET + Small_Losers_RET))/2
    return Portfolio_Return

WML = pd.DataFrame(data={'WML':[WML_VW(i,2) for i in formation_dates]},index=returns_dates)

# Real WML Downloaded from Ken French's Website

In [7]:
FFWML = mf.read('WRDS Data','F-F_Momentum_Factor',index_col=0,header=11)[469:1152]
FFWML.index = pd.to_datetime(FFWML.index,format='%Y%m')
FFWML = FFWML.apply(pd.to_numeric,errors='coerce')
FFWML = FFWML.resample('M').last()
FFWML.iloc[:,0]/=100
FFWML.head(4)
combined = pd.concat([FFWML,WML],axis=1)
combined.columns = ['Real','Replication']
round(combined,4)[:-36]

,Real,Replication
1966-02-28,0.0457,0.0493
1966-03-31,0.0142,0.0151
1966-04-30,0.0628,0.0589
1966-05-31,-0.0470,-0.0507
1966-06-30,0.0325,0.0309
...,...,...
2019-08-31,0.0692,0.0712
2019-09-30,-0.0681,-0.0615
2019-10-31,0.0015,0.0041
2019-11-30,-0.0266,-0.0204


In [8]:
# Correlation Between The Replication and The Real
print('Correlation Coefficent:',round(combined.corr().iloc[0,1],3))

Correlation Coefficent: 0.983
